In [1]:
!pip install datasets

In [2]:

from huggingface_hub import login

In [3]:
login()

In [4]:
from datasets import load_dataset

ds = load_dataset("iTzMiNOS/voice-orders-small-clean-12k")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['transcribed_text', 'speaker', 'items', 'count', 'audio'],
        num_rows: 12000
    })
})

In [6]:
ds["train"][0]

{'transcribed_text': "I'll have a Cream of Mushroom with truffle oil and chopped parsley, a Greek salad without protein, add grilled chicken and falafel, a vegan burger with mustard, ketchup, no cheese, vegan cheese, and vegan mayo, plus extra cheese on the side.",
 'speaker': 'af_bella',
 'items': [{'customizations': ['Truffle Oil', 'Chopped Parsley'],
   'name': 'Cream of Mushroom'},
  {'customizations': ['No Protein', 'Grilled Chicken', 'Falafel'],
   'name': 'Greek Salad'},
  {'customizations': ['Mustard',
    'Ketchup',
    'No Cheese',
    'Vegan Cheese',
    'Vegan Mayo'],
   'name': 'Vegan Burger'},
  {'customizations': [], 'name': 'Extra Cheese'}],
 'count': 4,
 'audio': {'path': 'response_0.wav',
  'array': array([-3.05175781e-05,  0.00000000e+00, -3.05175781e-05, ...,
         -3.05175781e-05,  0.00000000e+00,  0.00000000e+00]),
  'sampling_rate': 24000}}

In [7]:
!pip install torchaudio accelerate jiwer

In [8]:
distinct_speakers_hf = ds["train"].unique('speaker')
distinct_speakers_hf

['af_bella',
 'af_heart',
 'af_nicole',
 'am_fenrir',
 'am_michael',
 'am_puck',
 'bf_emma',
 'bf_isabella',
 'bm_fable',
 'bm_george']

In [9]:
from datasets import Dataset

total_rows = len(ds["train"])
split_size = 1200
num_datasets = 10

if total_rows != num_datasets * split_size:
    raise ValueError(f"Total rows ({total_rows}) is not divisible by the split size ({split_size}).")

datasets = []
for i in range(num_datasets):
    start_index = i * split_size
    end_index = (i + 1) * split_size
    ds_temp = ds["train"].select(range(start_index, end_index))
    datasets.append(ds_temp)


ds_1 = datasets[0]
ds_2 = datasets[1]
ds_3 = datasets[2]
ds_4 = datasets[3]
ds_5 = datasets[4]
ds_6 = datasets[5]
ds_7 = datasets[6]
ds_8 = datasets[7]
ds_9 = datasets[8]
ds_10 = datasets[9]

print(f"Number of rows in ds_1: {len(ds_1)}")
print(f"Number of rows in ds_10: {len(ds_10)}")

Number of rows in ds_1: 1200
Number of rows in ds_10: 1200


In [10]:
print(ds_1.unique('speaker'),
ds_2.unique('speaker'),
ds_3.unique('speaker'),
ds_4.unique('speaker'),
ds_5.unique('speaker'),
ds_6.unique('speaker'),
ds_7.unique('speaker'),
ds_8.unique('speaker'),
ds_9.unique('speaker'),
ds_10.unique('speaker'))

['af_bella'] ['af_heart'] ['af_nicole'] ['am_fenrir'] ['am_michael'] ['am_puck'] ['bf_emma'] ['bf_isabella'] ['bm_fable'] ['bm_george']


In [11]:
datasets

[Dataset({
     features: ['transcribed_text', 'speaker', 'items', 'count', 'audio'],
     num_rows: 1200
 }),
 Dataset({
     features: ['transcribed_text', 'speaker', 'items', 'count', 'audio'],
     num_rows: 1200
 }),
 Dataset({
     features: ['transcribed_text', 'speaker', 'items', 'count', 'audio'],
     num_rows: 1200
 }),
 Dataset({
     features: ['transcribed_text', 'speaker', 'items', 'count', 'audio'],
     num_rows: 1200
 }),
 Dataset({
     features: ['transcribed_text', 'speaker', 'items', 'count', 'audio'],
     num_rows: 1200
 }),
 Dataset({
     features: ['transcribed_text', 'speaker', 'items', 'count', 'audio'],
     num_rows: 1200
 }),
 Dataset({
     features: ['transcribed_text', 'speaker', 'items', 'count', 'audio'],
     num_rows: 1200
 }),
 Dataset({
     features: ['transcribed_text', 'speaker', 'items', 'count', 'audio'],
     num_rows: 1200
 }),
 Dataset({
     features: ['transcribed_text', 'speaker', 'items', 'count', 'audio'],
     num_rows: 1200
 }),
 

In [12]:
from datasets import Audio

for i in range(len(datasets)):
  datasets[i] = datasets[i].remove_columns([col for col in ds["train"].column_names if col not in ["audio", "speaker", "transcribed_text"]])
  datasets[i] = datasets[i].rename_column("transcribed_text", "text")
  datasets[i] = datasets[i].cast_column("audio", Audio(sampling_rate=16000))
  datasets[i]

In [13]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

model_name = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

processor.tokenizer.set_prefix_tokens(language="english", task="transcribe")

In [14]:
def prepare_batch(batch):
    audio = batch["audio"]

    inputs = processor(
        audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_tensors="pt"
    )

    labels = processor.tokenizer(
        batch["text"],
        return_tensors="pt",
        padding="longest",
        truncation=True
    ).input_ids

    batch["input_features"] = inputs.input_features[0]
    batch["labels"] = labels[0]
    return batch

for i in range(len(datasets)):
  datasets[i] = datasets[i].map(prepare_batch, remove_columns=datasets[i].column_names, num_proc=1)

In [15]:
from dataclasses import dataclass
import torch
from typing import Any

@dataclass
class DataCollatorSpeechSeq2Seq:
    processor: Any

    def __call__(self, features):
        input_features = torch.stack([
            torch.tensor(f["input_features"]) for f in features
        ])

        labels = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(f["labels"]) for f in features],
            batch_first=True,
            padding_value=self.processor.tokenizer.pad_token_id
        )
        labels[labels == self.processor.tokenizer.pad_token_id] = -100
        return {"input_features": input_features, "labels": labels}

In [16]:
from datasets import concatenate_datasets
split_datasets = []
for i in range(len(datasets)):
  split_datasets.append(datasets[i].train_test_split(test_size=0.2, seed=42))

train_datasets = [sd['train'] for sd in split_datasets]
test_datasets = [sd['test'] for sd in split_datasets]

merged_train_dataset = concatenate_datasets(train_datasets)
merged_test_dataset = concatenate_datasets(test_datasets)

In [17]:
merged_train_dataset

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 9600
})

In [18]:
merged_test_dataset

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 2400
})

In [19]:
!pip install tensorboard
!pip install wandb

In [20]:
!pip install evaluate

In [21]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

''' training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    warmup_steps=500,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    eval_strategy="no",
    fp16=False,
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=processor.feature_extractor,
    data_collator=DataCollatorSpeechSeq2Seq(processor),
)


trainer.train() '''

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=500,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    eval_strategy="no",
    fp16=True,
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=processor.feature_extractor,
    data_collator=DataCollatorSpeechSeq2Seq(processor),
)

trainer.train()


NameError: name 'split_dataset' is not defined

In [23]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [27]:
import evaluate
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, WhisperProcessor, WhisperForConditionalGeneration
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

normalizer = BasicTextNormalizer()

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True, normalize=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True, normalize=True)

    filtered_pred_str = []
    filtered_label_str = []
    for pred_s, label_s in zip(pred_str, label_str):
        if len(label_s) > 0:
             filtered_pred_str.append(pred_s)
             filtered_label_str.append(label_s)

    if not filtered_label_str:
        print("Warning: All labels were empty after filtering. Returning WER = 1.0")
        wer = 1.0
    else:
        wer = wer_metric.compute(predictions=filtered_pred_str, references=filtered_label_str)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}
    return {"wer": wer}

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    warmup_steps=400,
    dataloader_num_workers=4,
    num_train_epochs=3,
    # Remove evaluation during training
    # eval_strategy="steps",
    # eval_steps=720,
    logging_steps=360,
    report_to=["tensorboard"],
    save_steps=720,
    save_total_limit=4,
    # load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    fp16=True,
    gradient_checkpointing=True
)

if training_args.gradient_checkpointing:
    model.config.use_cache = False

@dataclass
class DataCollatorSpeechSeq2Seq:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (
            labels.shape[-1] == batch["input_features"].shape[-1]
            and hasattr(self.processor.feature_extractor, "config")
            and hasattr(self.processor.feature_extractor.config, "max_source_positions")
            and batch["input_features"].shape[-1] > self.processor.feature_extractor.config.max_source_positions
        ):

             print("Warning: Input feature length might exceed max source positions.")
             pass


        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2Seq(processor=processor)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=merged_train_dataset,
    eval_dataset=merged_test_dataset,
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

results = trainer.evaluate()
print("Final Evaluation Results:", results)

<ipython-input-27-07abe9e9d4c0>:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
360,0.011500
720,0.015000
1080,0.007600
1440,0.001900
1800,0.000500
2160,0.000300
2520,0.000300
2880,0.000100
3240,0.000000
3600,0.000100


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.95 GiB. GPU 0 has a total capacity of 22.16 GiB of which 1.89 GiB is free. Process 87851 has 20.27 GiB memory in use. Of the allocated memory 16.41 GiB is allocated by PyTorch, and 3.62 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [37]:
!rm -rf whisper-small-opt-small-clean

In [32]:
test_model = model.from_pretrained("./whisper-small-finetuned/checkpoint-1500")

trainer = Seq2SeqTrainer(
    model=test_model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=processor.feature_extractor,
    data_collator=DataCollatorSpeechSeq2Seq(processor),
)

evaluation_results = trainer.evaluate(eval_dataset=split_dataset["test"])

print("Evaluation results:", evaluation_results)

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': './whisper-small-finetuned/checkpoint-1500'. Use `repo_type` argument if needed.

In [ ]:
test_model = model.from_pretrained("./whisper-small-finetuned/checkpoint-1000")

trainer = Seq2SeqTrainer(
    model=test_model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=processor.feature_extractor,
    data_collator=DataCollatorSpeechSeq2Seq(processor),
)



evaluation_results = trainer.evaluate(eval_dataset=split_dataset["test"])

print("Evaluation results:", evaluation_results)

<ipython-input-15-d531854cc8e0>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Evaluation results: {'eval_loss': 0.01593630760908127, 'eval_model_preparation_time': 0.006, 'eval_runtime': 193.2076, 'eval_samples_per_second': 6.211, 'eval_steps_per_second': 0.776}


In [38]:
from transformers import GenerationConfig

# Create a NEW GenerationConfig without forced_decoder_ids
new_config = GenerationConfig.from_dict(model.generation_config.to_dict())
new_config.forced_decoder_ids = None  # Explicitly remove
new_config._from_model_config = False  # Important for newer versions

# Apply the new config
model.generation_config = new_config

# Verify removal
print("Updated generation config:", model.generation_config)

# Save everything
trainer.save_model("./whisper-small-opt-small-clean")
processor.save_pretrained("./whisper-small-opt-small-clean")

# Verify the saved model
from transformers import WhisperForConditionalGeneration
loaded_model = WhisperForConditionalGeneration.from_pretrained("./whisper-small-opt-small-clean")
loaded_model.generation_config = new_config
print("Loaded model config:", loaded_model.generation_config)

Updated generation config: GenerationConfig {
  "alignment_heads": [
    [
      5,
      3
    ],
    [
      5,
      9
    ],
    [
      8,
      0
    ],
    [
      8,
      4
    ],
    [
      8,
      7
    ],
    [
      8,
      8
    ],
    [
      9,
      0
    ],
    [
      9,
      7
    ],
    [
      9,
      9
    ],
    [
      10,
      5
    ]
  ],
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "decoder_start_token_id": 50258,
  "eos_token_id": 50257,
  "is_multilingual": true,
  "lang_to_id": {
    "<|af|>": 50327,
    "<|am|>": 50334,
    "<|ar|>": 50272,
    "<|as|>": 50350,
    "<|az|>": 50304,
    "<|ba|>": 50355,
    "<|be|>": 50330,
    "<|bg|>": 50292,
    "<|bn|>": 50302,
    "<|bo|>": 50347,
    "<|br|>": 50309,
    "<|bs|>": 50315,
    "<|ca|>": 50270,
    "<|cs|>": 50283,
    "<|cy|>": 50297,
    "<|da|>": 50285,
    "<|de|>": 50261,
    "<|el|>": 50281,
    "<|en|>": 50259,
    "<|es|>": 50262,
    "<|et|>": 50307,
  

In [44]:
print(merged_test_dataset[0])
print(type(merged_test_dataset[0]['input_features']))

Output hidden; open in https://colab.research.google.com to view.

In [46]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load

# 1. Load the pre-trained model and processor from Hugging Face Hub
model_id = "iTzMiNOS/whisper-small-finetuned-new"
processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id)
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")

# Your evaluation dataset is now correctly preprocessed
eval_dataset = merged_test_dataset

# 2. Generate predictions on the evaluation dataset
def generate_predictions(batch):
    # Convert the list of lists to a list of tensors
    input_features = [torch.tensor(features).squeeze(0) for features in batch["input_features"]]
    # Now stack the list of tensors
    input_features = torch.stack(input_features).to(model.device)
    with torch.no_grad():
        predicted_ids = model.generate(input_features)
    predicted_str = processor.batch_decode(predicted_ids, skip_special_tokens=True, normalize=True)
    # Return a dictionary with the predicted text
    return {"predicted_text": predicted_str}

predicted_transcriptions = eval_dataset.map(generate_predictions, batched=True, batch_size=8, remove_columns=eval_dataset.column_names)["predicted_text"]

# 3. Load evaluation metrics
wer = load("wer")
cer = load("cer")

# 4. Extract ground truth labels
ground_truth_transcriptions = processor.batch_decode(eval_dataset["labels"], skip_special_tokens=True, normalize=True)

# 5. Compute metrics
wer_score = wer.compute(predictions=predicted_transcriptions, references=ground_truth_transcriptions)
cer_score = cer.compute(predictions=predicted_transcriptions, references=ground_truth_transcriptions)

print(f"WER on the evaluation set: {wer_score}")
print(f"CER on the evaluation set: {cer_score}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

WER on the evaluation set: 0.0016144532000768787
CER on the evaluation set: 0.000499380330976161


In [48]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from evaluate import load
from datasets import Dataset

# Assuming you have your eval_dataset (Hugging Face Dataset object)
# and predicted_transcriptions, ground_truth_transcriptions lists

# 1. Create a new Dataset with predictions
results_dataset = Dataset.from_dict({
    "predicted_text": predicted_transcriptions,
    "ground_truth": ground_truth_transcriptions
})

# 2. Define a function to compute WER and CER
def compute_metrics(example):
    wer_metric = load("wer")
    cer_metric = load("cer")
    wer = wer_metric.compute(predictions=[example["predicted_text"]], references=[example["ground_truth"]])
    cer = cer_metric.compute(predictions=[example["predicted_text"]], references=[example["ground_truth"]])
    return {"wer": wer, "cer": cer}

# 3. Use the map function to efficiently compute metrics
computed_metrics = results_dataset.map(compute_metrics)

# 4. Convert the computed metrics to a Pandas DataFrame for plotting
df_metrics = computed_metrics.to_pandas()

# 5. Plotting WER and CER Distributions (Faster with Pandas/Seaborn)
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(df_metrics['wer'], bins=20, kde=True)
plt.title('Distribution of Word Error Rate (WER)')
plt.xlabel('WER')
plt.ylabel('Frequency')
plt.grid(True)

plt.subplot(1, 2, 2)
sns.histplot(df_metrics['cer'], bins=20, kde=True)
plt.title('Distribution of Character Error Rate (CER)')
plt.xlabel('CER')
plt.ylabel('Frequency')
plt.grid(True)

plt.tight_layout()
plt.show()

# 6. Plotting WER vs. Audio Length and CER vs. Audio Length (if available)
if 'audio_length' in eval_dataset.column_names:
    df_combined = df_metrics.merge(eval_dataset.to_pandas()[['audio_length']], left_index=True, right_index=True)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    sns.scatterplot(x='audio_length', y='wer', data=df_combined)
    plt.title('WER vs. Audio Length')
    plt.xlabel('Audio Length (seconds or frames)')
    plt.ylabel('WER')
    plt.grid(True)

    plt.subplot(1, 2, 2)
    sns.scatterplot(x='audio_length', y='cer', data=df_combined)
    plt.title('CER vs. Audio Length')
    plt.xlabel('Audio Length (seconds or frames)')
    plt.ylabel('CER')
    plt.grid(True)

    plt.tight_layout()
    plt.show()
else:
    print("Audio length information not found in the dataset. Skipping WER/CER vs. Audio Length plots.")

# Alternative Plot Design: Box Plots
plt.figure(figsize=(8, 6))
sns.boxplot(y=df_metrics['wer'])
plt.title('Box Plot of Word Error Rate (WER)')
plt.ylabel('WER')
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 6))
sns.boxplot(y=df_metrics['cer'])
plt.title('Box Plot of Character Error Rate (CER)')
plt.ylabel('CER')
plt.grid(True)
plt.show()

print("Plots generated.")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
import shutil

shutil.make_archive('./whisper-small-finetuned-tokenizer', 'zip', './whisper-small-opt-small-clean')

'/content/whisper-small-finetuned-tokenizer.zip'

In [ ]:
from google.colab import files

files.download('./whisper-small-finetuned-1000.zip')
files.download('./whisper-small-finetuned-1500.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
import shutil

# Define the directory you want to delete
directory = './whisper-small-finetuned/checkpoint-1500'

# Remove all files inside the directory first
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)

    # If it's a file, remove it
    if os.path.isfile(file_path):
        os.remove(file_path)

    # If it's a directory, remove it recursively
    elif os.path.isdir(file_path):
        shutil.rmtree(file_path)

# Finally, remove the empty directory
os.rmdir(directory)


In [39]:
from huggingface_hub import HfApi

repo_name = "iTzMiNOS/whisper-small-finetuned-new"

api = HfApi()

api.upload_folder(
    folder_path="./whisper-small-opt-small-clean",
    repo_id=repo_name,
    repo_type="model",
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/iTzMiNOS/whisper-small-finetuned-new/commit/1e5da4c190b75a67396d70569f2d90e8cfb038f5', commit_message='Upload folder using huggingface_hub', commit_description='', oid='1e5da4c190b75a67396d70569f2d90e8cfb038f5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/iTzMiNOS/whisper-small-finetuned-new', endpoint='https://huggingface.co', repo_type='model', repo_id='iTzMiNOS/whisper-small-finetuned-new'), pr_revision=None, pr_num=None)

In [ ]:
from IPython.display import display, Javascript
import torch
import librosa
from google.colab import files
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 1. Record Audio in Colab
def record_audio():
    display(Javascript('''
        async function record() {
            var stream = await navigator.mediaDevices.getUserMedia({audio: true});
            var recorder = new MediaRecorder(stream);
            var audioChunks = [];
            recorder.ondataavailable = event => audioChunks.push(event.data);
            recorder.onstop = () => {
                var audioBlob = new Blob(audioChunks);
                var audioUrl = URL.createObjectURL(audioBlob);
                var audio = new Audio(audioUrl);
                audio.controls = true;
                document.body.appendChild(audio);

                // Save the audio file in a downloadable format
                var link = document.createElement("a");
                link.href = audioUrl;
                link.download = "recorded_audio.wav";  // Name the file here
                document.body.appendChild(link);
                link.click();  // Automatically click the link to trigger download
            };
            recorder.start();
            setTimeout(() => recorder.stop(), 15000);  // Record for 10 seconds
        }
        record();
    '''))

# 2. Transcribe Recorded Audio (Once File is Downloaded)
def transcribe_audio(audio_path):
    # Load the audio file
    audio, sampling_rate = librosa.load(audio_path, sr=16000)

    # Prepare the audio for the model
    inputs = processor(audio, sampling_rate=sampling_rate, return_tensors="pt")

    # Generate transcription
    with torch.no_grad():
        generated_ids = model.generate(inputs.input_features)

    # Decode the generated ids to text
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return transcription

In [ ]:
record_audio()

<IPython.core.display.Javascript object>

In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-small")

# Save the tokenizer
processor.save_pretrained("./whisper-small-finetuned")

[]

"I'll take some spicy coleslaw, mozzarella sticks with marinara and ranch, onion rings with ranch and BBQ, grilled tofu with peanut sauce, soy ginger, and chili lime, orange fresh juice, a selection of rose, white, and red wine, and extra whipped cream on the side."

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor, GenerationConfig
import librosa
import torch

new_config = GenerationConfig.from_dict(model.generation_config.to_dict())
new_config.forced_decoder_ids = None
new_config._from_model_config = False

processor = WhisperProcessor.from_pretrained("itzminos/whisper-small-finetuned")

mymodel = WhisperForConditionalGeneration.from_pretrained("itzminos/whisper-small-finetuned")
mymodel.generation_config = new_config

audio_path = "/content/recaudio.wav"
audio_array, sampling_rate = librosa.load(audio_path, sr=16000)

inputs = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt").input_features.to("cuda:0")

outputs = model.generate(inputs)
transcription = processor.batch_decode(outputs, skip_special_tokens=True)[0]

print("Transcription:", transcription)

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

<ipython-input-31-6369a50c3a16>:15: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_array, sampling_rate = librosa.load(audio_path, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obta

Transcription: I'll take some spicy coleslaw, mozzarella sticks with marinara and ranch, onion rings with ranch and BBQ, grilled tofu with peanut sauce, soy ginger, and chili lime, orange fresh juice, a selection of rose, white, and red wine, and extra whipped cream on the side. ________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [ ]:
text = "I'll have a Garden Salad with vinaigrette and balsamic, mozzarella sticks with ranch and marinara, crispy fish and chips, a side of strawberries, extra dipping sauces with blue cheese and ranch, extra sauces with gravy, BBQ sauce, and pesto, and some extra toppings with nuts and berries."

In [ ]:
text

"I'll have a Garden Salad with vinaigrette and balsamic, mozzarella sticks with ranch and marinara, crispy fish and chips, a side of strawberries, extra dipping sauces with blue cheese and ranch, extra sauces with gravy, BBQ sauce, and pesto, and some extra toppings with nuts and berries."

In [ ]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 14.6 MB/s eta 0:00:00


In [ ]:
import Levenshtein

str1 = "kitten"
str2 = "sitting"

# Calculate Levenshtein distance
distance = Levenshtein.distance(text, transcribe)
print(f"Levenshtein distance: {distance}")  # Output: 3

# Calculate similarity ratio (0.0 to 1.0)
similarity_ratio = Levenshtein.ratio(text, transcribe)
print(f"Similarity ratio: {similarity_ratio:.2f}")  # Output: 0.62

Levenshtein distance: 6
Similarity ratio: 0.98


In [ ]:
from transformers import pipeline, WhisperForConditionalGeneration, WhisperProcessor
import torch

processor = WhisperProcessor.from_pretrained("itzminos/whisper-tiny-finetuned-basic")

new_config = GenerationConfig.from_dict(model.generation_config.to_dict())
new_config.forced_decoder_ids = None
new_config._from_model_config = False

processor = WhisperProcessor.from_pretrained("itzminos/whisper-tiny-finetuned-basic")

model = WhisperForConditionalGeneration.from_pretrained("itzminos/whisper-tiny-finetuned-basic")
model.generation_config = new_config

transcriber = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
)

audio_path = "response_3001.wav"
result = transcriber(audio_path)
print("Transcription:", result["text"])

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcription:  Can I get an extra scoop of ice cream, a vegan burger with no cheese, vegan mayo, and mustard, an apple colt, a Greek salad with falafel, oil turkey with cheddar cheese, a fruit salad, a garden salad with vinaigrette ranch and balsamic, spring rolls with vegetarian and chicken, extra BBQ sauce and pasta, and a cheesecake?


In [ ]:
text = "I'll take some spicy coleslaw, mozzarella sticks with marinara and ranch, onion rings with ranch and BBQ, grilled tofu with peanut sauce, soy ginger, and chili lime, orange fresh juice, a selection of rose, white, and red wine, and extra whipped cream on the side."
trans = "I'll take some spicy coleslaw, mozzarella sticks with marinara and ranch, onion rings with ranch and BBQ, grilled tofu with peanut sauce, soy ginger, and chili lime, orange fresh juice, a selection of rose, white, and red wine, and extra whipped cream on the side."
from jiwer import wer

error_rate = wer(text.lower(), trans.lower())
accuracy = (1 - error_rate) * 100
print(f"WER: {error_rate:.2f}, Accuracy: {accuracy:.2f}%")

WER: 0.00, Accuracy: 100.00%
